<a href="https://colab.research.google.com/github/Yuxin19/Text-Social-Media-Analysis/blob/master/TWSM_E4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# load pacakges
!pip install transformers
import pickle
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from google.colab import files
from scipy.spatial.distance import cosine
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import *
import numpy as np
from scipy.spatial.distance import cdist

     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 3.8MB 14.7MB/s 
     |████████████████████████████████| 1.1MB 30.9MB/s 
     |████████████████████████████████| 890kB 38.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f7afd36e84a2de2b99a540fb95fff53e0117f32bc505a2a1ed47bca7f45ff41a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
uploaded = files.upload()

Saving stemmed.pkl to stemmed.pkl


In [0]:
#loading the stemmed data with pickle
with open('stemmed.pkl', 'rb') as file:
    data_stem = pickle.load(file)

#Printing first entry for inspection
print(data_stem[0])

car wonder enlighten car saw dai door sport car look late earli call bricklin door small addit bumper separ rest bodi know tellm model engin spec year product car histori info funki look car mail thank


In [0]:
#Creation of corpus with gensim
corpus_gen = [doc.split() for doc in data_stem]

# Part 1: Word2vec

### 5. Run the following code. What is it doing? 

It firstly initializes the the model, then applies the embeddings to the corpus with Word2Vec(). 

* Word2Vec is a more recent model that embeds words in a lower-dimensional vector space using a shallow neural network. The result is a set of word-vectors where vectors close together in vector space have similar meanings based on context, and word-vectors distant to each other have differing meanings. For example, strong and powerful would be close together and strong and Paris would be relatively far. (source: gesim Word2Vec Model)

* The ***size*** argument defines the dimensionality of the of the vectors and they are usually much smaller than the amount of words in the vocabulary. 

* The ***min_count*** defines the minimum frequency for a word to be considered.

In [0]:
# Initializing the semi-trained model
path = get_tmpfile("word2vec.model")

In [0]:
# Applying the word2vec vectorization in the corpus
model = Word2Vec(corpus_gen, size=100, min_count=566)

In [0]:
# Saving the model trained on the corpus
model.save("word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### 6. Printing the sorted features of the model

* ***model.wv*** switches to the keyed vectors instance, which is a faster object 
* **vocabulary** is like a a dictionary with all the features, which here are the **keys**

In [0]:
sort_keys = model.wv.vocab.keys()
print(sort_keys)
# More features than in the previous exercise
print("Number of features:{0}".format(len(sort_keys)))

dict_keys(['car', 'wonder', 'saw', 'dai', 'look', 'call', 'small', 'addit', 'bodi', 'know', 'model', 'engin', 'year', 'product', 'histori', 'info', 'mail', 'thank', 'final', 'report', 'number', 'experi', 'send', 'messag', 'speed', 'rate', 'card', 'disk', 'function', 'request', 'network', 'base', 'answer', 'gui', 'washington', 'edu', 'question', 'mac', 'start', 'life', 'wai', 'new', 'machin', 'bit', 'mayb', 'anybodi', 'expect', 'heard', 'suppos', 'appear', 'access', 'price', 'line', 'like', 'went', 'recent', 'displai', 'probabl', 'got', 'feel', 'better', 'great', 'good', 'opinion', 'peopl', 'us', 'take', 'size', 'monei', 'hit', 'activ', 'real', 'subject', 'plai', 'figur', 'actual', 'help', 'perform', 'advanc', 'email', 'post', 'read', 'time', 'truth', 'uucp', 'wrote', 'write', 'articl', 'graphic', 'chip', 'far', 'low', 'level', 'stuff', 'pretti', 'nice', 'command', 'requir', 'point', 'address', 'phone', 'inform', 'com', 'system', 'thing', 'person', 'sens', 'world', 'clear', 'memori', 'e

### 7. What is ```model.wv['car']``` doing? Why does it have this size?
* wv object essentially contains the mapping between words and embeddings.
* we defined embedding vector as size 100 when initialing the Word2Vec model.

In [0]:
model.wv['car']

array([-0.26794216, -1.5041773 ,  1.9400327 ,  0.12434263,  1.0789795 ,
       -0.16389957,  0.67728674,  0.5973391 , -1.0873423 , -0.86074835,
        1.3373089 ,  0.419944  , -0.5970222 ,  0.32833105,  1.6259793 ,
       -0.41429228, -0.04742363, -1.6585678 ,  1.2377304 ,  0.764478  ,
       -0.05946701,  0.14410263,  0.10724014,  0.7793004 ,  0.2748657 ,
       -2.566935  ,  0.34753868, -1.7996372 ,  0.536633  ,  0.72741205,
       -1.1048667 ,  0.02959595, -0.48311612, -0.73700243, -0.6040208 ,
        0.6043265 ,  0.85392   ,  0.8697038 , -0.5825424 ,  0.43473402,
        0.3309866 ,  0.7618902 ,  1.5854511 , -0.52686435,  0.47770378,
        0.9562685 ,  0.4316423 ,  1.0250399 ,  0.12696682, -0.10763932,
       -1.1090778 , -1.4335455 ,  0.6863321 , -0.82881045,  0.86871445,
       -1.0476975 , -0.41775578, -1.4059968 , -0.602187  ,  1.0623872 ,
        0.95516014, -1.5638278 , -0.6668353 , -0.92872924,  1.2023159 ,
       -0.71700716, -0.25090688, -0.6502368 ,  1.4783542 , -0.58

###  8. Does the result of ```model.wv.most_similar('car')``` make sense?
* ```most_similar()```: Find the top-N most similar entities. 
* In the result there are words that are similar in meaning to "car", like "bike", or that obviously have something to do with it, like "road" or "speed". The others could probably appear together with car in a sentence.
* But there are two words that don't exist, because of stemming.

In [0]:
model.wv.most_similar('car')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bike', 0.6558433771133423),
 ('bui', 0.6312383413314819),
 ('road', 0.5628459453582764),
 ('nice', 0.4985658824443817),
 ('speed', 0.48535382747650146),
 ('price', 0.4731811285018921),
 ('pai', 0.4566709101200104),
 ('light', 0.44934147596359253),
 ('fast', 0.4489366114139557),
 ('friend', 0.44584232568740845)]

### 9. What is ```model.wv.most_similar(positive=['bike', 'engin'], topn=1)``` doing? Test other combinations.
* **most_similar** method finds the top-N most similar words, based on cosine similarity. Positive words contribute positively towards the similarity, negative words negatively.


In [0]:
# find the top-N most similar entities. 
model.wv.most_similar(positive=['bike', 'engin'], topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('car', 0.6569557189941406)]

In [0]:
model.wv.most_similar(positive=['american', 'arab'], topn=3)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('isra', 0.7852672338485718),
 ('israel', 0.7105824947357178),
 ('countri', 0.7020965814590454)]

In [0]:
#Whaaaaat?
model.wv.most_similar(negative=['nice'], topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('relat', 0.5460495948791504)]

In [0]:
model.wv.most_similar(positive=['nice'], topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('pretti', 0.7642534971237183)]

### 10. Corpus as a list

1st step: List of vectors by document & getting rid of empty documents

In [0]:
corpus_list = []
index_list = []
for i, doc  in enumerate(corpus_gen):  # loop through docs in corpus

  doc_list = []

  for t in corpus_gen[i]:              # loop through words in document

    try:                               # Not all words from the corpus are in the vocabulary, so we try, otherwise pass
      doc_list.append(model.wv[t])     # appends vector to the list for that document
    except:
      pass

  if len(doc_list) != 0:               # If document is not empty
    corpus_list.append(doc_list)       # append document list to corpus list
    index_list.append(i)               # store its index
  else:
    print(i)                           # print documents that were empty

800
1015
1410
1636
3682
4474
4829
6294
6303
6324
7709
8076
9622
9989


2nd step: Aggregating over the docs by averaging the word vectors

In [0]:
ave_corpus_list = [sum(d) / len(d) for d in corpus_list]

In [0]:
# we originally have 11314 documents
print(len(ave_corpus_list))
print(len(index_list))

11300
11300


3rd step: Saving list in a df

* Each dimension of the vector as a feature 
* Original index for documents

In [0]:
# storing in a DF
word2vec_df = pd.DataFrame(ave_corpus_list, index=index_list)
word2vec_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-0.133520,-0.205871,0.540827,0.201590,0.228581,-0.188586,0.250787,0.140166,-0.231416,-0.284276,0.448701,0.024884,-0.109021,0.317746,0.479404,-0.186744,-0.089540,-0.470976,0.536832,0.424950,0.282519,-0.055702,0.200786,0.277524,0.526358,-0.765006,0.188224,-0.409778,0.057098,0.310174,-0.174347,-0.123835,-0.106624,-0.187790,-0.031910,0.226284,0.011690,0.277758,-0.345645,0.245592,...,0.192882,-0.611642,-0.200534,-0.187889,0.083978,-0.245766,-0.143312,-0.121288,0.205845,-0.013488,-0.079314,-0.022406,-0.246403,-0.295343,0.657918,0.595113,-0.053183,-0.464550,0.004884,0.204728,-0.026398,-0.355012,0.308426,-0.369635,0.396451,0.428443,0.136066,-0.161944,-0.154808,-0.280140,0.217243,-0.153772,-0.432498,0.608670,0.267975,0.108980,0.068954,0.328536,-0.107756,-0.393615
1,0.169566,-0.079456,0.086126,0.367178,-0.121486,-0.155744,0.084904,0.199997,0.028355,-0.421021,-0.312348,-0.094708,-0.201633,0.026161,0.203495,-0.402973,-0.080606,-0.206762,-0.038001,-0.068519,0.133714,-0.181731,0.421341,0.081757,0.380098,0.363073,0.211633,-0.336953,-0.273911,0.032675,0.188112,0.326032,-0.068985,0.183027,0.012522,0.031701,0.027768,0.021980,-0.185054,0.152063,...,0.019761,-0.299428,-0.024324,-0.196268,-0.276234,0.344797,-0.137360,0.018294,0.083729,-0.160569,0.134363,0.185317,-0.465967,0.169283,0.146258,0.341492,0.190334,-0.258629,-0.145917,-0.179967,-0.076148,0.072885,0.005028,0.005920,0.032668,0.113612,-0.111683,-0.291292,-0.144732,-0.379654,0.020979,0.017339,-0.229037,0.146866,0.435583,-0.055469,0.318941,0.194363,0.102507,-0.416049
2,-0.038571,-0.109795,0.265438,0.238047,-0.067245,0.041680,-0.053258,-0.050708,-0.098066,-0.217658,0.168907,-0.258018,-0.169833,0.155888,0.105353,-0.204443,0.079928,-0.247902,0.243638,0.140911,0.027223,-0.146578,0.211855,0.175544,0.523510,0.067215,0.133645,-0.145413,-0.138054,0.100622,0.047281,-0.007336,-0.045683,0.275633,-0.002559,0.088560,-0.016204,-0.034076,-0.041215,-0.091803,...,-0.012305,-0.222658,0.035143,-0.197591,-0.168372,0.103934,0.203879,-0.181492,-0.262982,0.156043,-0.029133,-0.055327,-0.213674,0.011754,0.208340,0.161101,-0.122102,-0.364265,-0.047563,-0.031674,-0.135330,0.129290,-0.008915,-0.093275,-0.066901,0.138251,-0.014481,0.117598,-0.047514,-0.136666,-0.062683,0.114591,-0.225952,0.071785,0.211907,0.187273,0.079857,-0.104236,-0.046201,-0.096563
3,0.230047,0.271594,0.189419,-0.144443,-0.336042,-0.042169,0.262576,-0.116351,-0.115010,-0.069672,0.052719,-0.123211,-0.233378,0.057986,0.217719,-0.100583,-0.023848,-0.075299,-0.128680,0.047721,-0.136950,0.084620,0.108962,-0.021866,0.064454,0.011272,0.067024,-0.075289,-0.017998,-0.057100,-0.111417,0.214996,-0.237404,0.267877,0.280718,-0.285532,0.266106,-0.125921,-0.155077,-0.365090,...,-0.113812,-0.510462,-0.196848,-0.121889,-0.209525,-0.009737,-0.173493,-0.144078,-0.191104,0.120993,-0.269951,0.105311,-0.256485,0.357902,0.252792,0.200386,0.312016,0.004721,-0.409952,0.024608,-0.289505,-0.142026,-0.057477,0.101723,0.428129,0.294926,-0.067074,-0.051613,-0.004120,-0.125989,-0.217658,0.201641,0.054438,-0.049543,0.148464,0.092256,0.182076,0.002059,0.147390,0.025817
4,0.113897,0.170048,0.108536,0.044734,-0.402438,0.197276,0.084575,-0.418756,-0.531716,-0.032977,-0.150673,-0.246992,-0.461042,-0.148635,-0.191908,-0.169503,0.046279,-0.270890,-0.103862,0.190700,-0.184368,0.036457,-0.158507,-0.220387,-0.094299,0.352715,0.106913,-0.151588,-0.259917,0.018320,-0.056084,0.138995,-0.087584,0.245539,-0.028197,-0.418746,0.220055,-0.214267,0.015969,-0.343842,...,-0.144638,-0.378051,0.023373,-0.236774,-0.088477,0.227137,0.104261,-0.192432,-0.193172,0.162747,0.102860,-0.010711,-0.298796,0.335166,-0.069963,0.012989,0.026566,0.191757,-0.359317,-0.065898,-0.185498,0.184495,-0.060726,0.160829,0.002626,0.066756,-0.107818,-0.126006,-0.121729,0.103222,-0.213828,0.114968,

### 11. Finding words similar to the first document's embedding

In [0]:
# most similar word to first document
model.wv.similar_by_vector(word2vec_df.iloc[0,:].to_numpy(), topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('car', 0.8725012540817261)]

Note: we used cdist to get cosine similarities instead of cosine()
* Computes distance between each pair of the two collections of inputs.

In [0]:
# most similar document to first document
docs_cosine_similarities = cdist(word2vec_df[:1], # First document
                                 word2vec_df,     # All other documents
                                 metric='cosine') # Cosine distance as metric
docs_cosine_similarities

array([[2.22044605e-16, 6.44993695e-01, 4.81061427e-01, ...,
        8.76546636e-01, 1.01554147e+00, 4.79950492e-01]])

In [0]:
# check cdist returns same result as cosine -> yes
print(cosine(word2vec_df[:1], word2vec_df[1:2]))
print(cosine(word2vec_df[:1], word2vec_df[2:3]))

0.6449936945727454
0.4810614274284857


In [0]:
print(np.argmax(docs_cosine_similarities)) # Returns the indices of the maximum values along an axis.

7920


In [0]:
most_similar_i = word2vec_df.index[np.argmax(docs_cosine_similarities)]
print(most_similar_i)
print(data_stem[most_similar_i])

7931
gritz jb liberti lobbi larouch christic insitut libertarian note gritz populist parti ticket david duke veep duke lead ticket withdrew candidaci gritz gave chanc vice presid aviod support duke


# Part 2: Doc2vec
* With the Word2Vec model, we can calculate the vectors for each word in a document. But what if we want to calculate a vector for the entire document? We could average the vectors for each word in the document - while this is quick and crude, it can often be useful. However, there is a better way -> Paragraph Vector. In Gensim, we refer to the Paragraph Vector model as Doc2Vec. (source: gesim Doc2Vec Model)

In [0]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus_gen)]
model = Doc2Vec(documents, vector_size=100, min_count=566)

In [0]:
model.save("doc2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# find most similar doc to the first doc
similar_doc = model.docvecs.most_similar(0, topn=1)
print(similar_doc)

[(958, 0.9110967516899109)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
print(data_stem[similar_doc[0][0]])

car articl apr wam umd edu lerxst wam umd edu thing write wonder enlighten car saw dai door sport car look late earli call bricklin door small addit bumper separ rest bodi know tellm model engin spec year product car histori info funki look car mail bricklin manufactur engin ford odd look encas bumper aren lot hem motor new usus list basic perform ford new style slap brought neighborhood lerxst rush fan robert seymour rseymour reed edu physic philosophi reed colleg nextmail accept artifici life project reed colleg reed solar energi project soltrain portland


In [0]:
doc2vec_df = pd.DataFrame()
for i in range(len(model.docvecs)):
  doc2vec_df = doc2vec_df.append(pd.Series(model.docvecs[i]), ignore_index=True)

In [0]:
doc2vec_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.009629,-0.013171,0.033003,0.000803,0.030401,0.035090,-0.001415,-0.038519,0.000959,-0.049577,0.035694,0.002966,-0.008271,0.052822,0.020462,-0.019862,0.012552,-0.023000,0.019125,0.020253,0.023148,-0.002817,0.025489,0.013015,0.110589,-0.082568,0.021161,-0.035227,0.011016,0.013371,0.002564,-0.002693,-0.014906,0.038598,0.005644,0.038629,0.009147,0.038484,-0.042424,0.030187,...,0.046050,-0.032183,-0.019468,-0.013069,-0.033569,-0.004999,-0.044485,-0.016654,0.027465,-0.004784,-0.022423,-0.092259,-0.056320,-0.016811,0.091301,0.077009,-0.033287,-0.018261,-0.021801,0.003477,-0.002297,-0.047647,0.031138,-0.073916,0.052414,0.055391,0.009311,-0.001102,0.038236,-0.008581,-0.004393,-0.004060,-0.037001,0.055227,0.015121,0.036837,0.051495,0.040835,0.047040,-0.035837
1,0.044619,-0.007349,0.004707,0.034858,0.001797,0.010287,-0.002642,-0.000785,0.009791,-0.005372,-0.054048,0.011480,-0.000948,0.026023,0.019384,-0.003925,-0.018162,0.002569,-0.010419,-0.014990,0.028530,-0.013560,0.013671,-0.036985,0.063446,0.026071,0.036112,0.010715,-0.033669,0.003892,0.051523,-0.000105,0.022249,0.039073,0.008241,0.011200,0.007054,0.000476,-0.026620,0.002351,...,0.007714,0.012207,-0.016511,-0.006621,-0.016302,0.029133,-0.010425,0.013894,0.026590,-0.025717,0.031962,-0.029603,-0.045437,0.018666,0.043409,0.018956,0.005192,0.024315,-0.039599,-0.003963,-0.011017,-0.027100,-0.006565,-0.020444,0.000994,-0.012205,-0.017111,-0.044053,-0.002164,-0.007170,0.013803,-0.002470,-0.018794,0.019451,0.077577,-0.010800,0.029991,0.060825,0.021231,-0.032885
2,0.035321,-0.014300,0.074565,0.035204,0.007065,-0.002703,-0.039654,0.022884,0.012082,-0.084308,0.083282,-0.054857,-0.027645,0.006281,0.080074,-0.026019,-0.025068,-0.083999,-0.001013,0.012543,-0.064191,0.040882,0.064510,-0.016387,0.046947,0.013639,0.019946,-0.010987,-0.038555,0.058442,0.024750,0.040023,0.014814,0.014379,0.024260,0.028303,0.011145,-0.001805,0.015604,0.042603,...,0.020899,-0.053390,-0.018836,-0.035867,-0.035831,0.035409,0.085296,-0.110204,-0.062743,-0.027649,-0.002670,-0.019828,-0.095519,-0.002749,0.053339,0.024153,-0.013485,-0.097991,0.015014,0.023152,0.041359,0.097941,0.027598,-0.044002,-0.083870,0.019530,0.000084,0.104387,-0.024864,-0.087476,-0.008168,-0.007816,-0.057808,-0.007096,0.080731,0.045912,-0.022241,-0.049585,-0.041141,0.002441
3,-0.012454,0.025048,0.002345,-0.008606,-0.011133,0.029840,0.015058,-0.014074,-0.020769,-0.024252,0.002253,-0.019211,-0.049352,-0.005404,0.032538,-0.055659,0.023471,0.014878,-0.013451,-0.019312,-0.040459,-0.004707,0.037162,0.028556,0.022780,-0.005330,0.019203,-0.020651,-0.004116,-0.033284,-0.042438,0.009668,-0.032442,0.029184,0.033520,-0.044695,0.005925,-0.006127,-0.035749,-0.018095,...,0.042673,-0.054943,-0.015287,-0.047460,-0.005924,-0.026373,0.015209,-0.021495,-0.025737,0.009910,-0.050136,-0.003620,-0.027087,0.043935,0.026502,0.009717,0.022136,-0.014205,-0.022137,0.025603,-0.024987,-0.019736,-0.005427,-0.006290,-0.003477,0.021121,-0.012578,-0.001741,0.000222,-0.001337,-0.021058,0.007528,0.031422,-0.014133,-0.041667,0.041517,0.022856,-0.013208,0.004821,-0.010830
4,-0.019443,0.023501,0.020024,0.019612,-0.026380,0.051271,-0.009834,-0.076098,-0.110022,-0.038446,-0.019326,-0.053769,-0.093304,-0.028101,-0.020350,-0.080887,0.020875,-0.050403,0.016471,0.066683,-0.032394,0.010264,0.000055,0.016506,0.011057,0.064328,0.040789,-0.064476,-0.034076,-0.002521,-0.024999,-0.012850,-0.036766,0.095997,-0.080622,-0.028790,0.041510,0.009221,-0.011511,-0.056819,...,-0.029571,-0.039982,-0.033121,-0.099368,0.009381,-0.001511,0.057263,-0.034565,-0.034019,0.052853,0.001239,-0.010567,-0.061849,0.097649,0.003026,0.004524,-0.012056,0.021972,-0.069608,-0.015697,-0.050692,-0.006897,0.003038,0.005375,0.001638,-0.027451,-0.013207,-0.052468,-0.008100,0.015485,-0.056612,0.022499,-0

# Part 3: ELMo

In [0]:
# load eomo model from TensorFlow Hub
elmo = hub.Module("https://tfhub.dev/google/elmo/3")

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/3'.
INFO:absl:Downloaded https://tfhub.dev/google/elmo/3, Total size: 357.40MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/3'.


elmo parameters
* With the ***default signature***, the module takes ***untokenized sentences*** as input. The input tensor is a string tensor with shape [batch_size]. The module tokenizes each string by splitting on spaces.
* **['default']**: a fixed mean-pooling of all contextualized word representations with shape [batch_size, 1024]

In [0]:
embeddings = elmo(data_stem[0:10], signature="default", as_dict=True)["default"]
embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([10, 1024])

In [0]:
# run the the model. You can not see the content of the tensor without running the graph (doing session.run()).
# using with to prevent forgetting close the session
with tf.compat.v1.Session() as sess:
  sess.run(tf.compat.v1.global_variables_initializer())
  corpus_elm = sess.run(embeddings)

### Print the embedding values for the first document. Can you interpret it?
* it is a fixed mean-pooling of all contextualized word representations

In [0]:
print(corpus_elm[0])

[ 0.07138149  0.1951943  -0.09149864 ... -0.37476012  0.1886747
  0.16174579]


#### How can you use corpus_elm_df for further tasks?
You can use it whenever you have to vectorize text data.
* Classification
* Text Summarization
* Named Entity Recognition
* Question-Answering Systems

In [0]:
corpus_elm_df = pd.DataFrame(corpus_elm)
corpus_elm_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.071381,0.195194,-0.091499,-0.056139,0.060336,0.602086,-0.428267,0.147217,-0.223365,-0.002364,-0.360357,0.267460,-0.653219,0.014107,-0.011198,0.153389,0.378626,0.386909,-0.418051,0.069700,0.755093,-0.149906,0.002108,-0.310571,0.213515,-0.101479,0.197859,-0.121517,0.207179,0.278058,0.341266,0.185140,0.188046,-0.467464,0.003810,0.232802,-0.347815,0.272836,0.354792,0.684967,...,0.354261,0.281943,0.001335,0.397741,0.297757,-0.502097,0.096041,0.063876,0.032441,-0.173737,0.188458,0.047619,-0.381535,-0.310682,-0.101096,0.348021,0.140095,-0.000057,-0.233020,0.182884,0.396763,-0.139555,0.102448,-0.637644,-0.141189,-0.086753,-0.182304,0.053941,-0.247799,-0.055875,-0.276381,0.332053,-0.225546,0.123051,-0.108826,-0.335023,0.275925,-0.374760,0.188675,0.161746
1,0.197677,0.139034,-0.162011,-0.094836,0.130265,0.229831,-0.285738,0.421668,0.197578,-0.164257,-0.247900,0.047955,-0.567297,0.121708,0.165591,0.036047,0.281482,0.446328,-0.587396,-0.068240,0.624128,0.427305,-0.050798,-0.060474,0.200220,0.143442,0.347401,-0.088222,0.098820,0.304108,-0.104149,0.164165,0.228108,-0.653909,0.032777,0.108331,-0.268706,0.167775,0.322713,0.587584,...,0.262288,0.285806,0.228223,0.475427,0.499776,-0.465738,0.265414,0.420376,0.114693,-0.120252,-0.058504,-0.070585,-0.316707,-0.141274,0.093630,0.424726,0.224331,0.442176,0.111860,0.140983,0.555419,-0.242645,0.113901,-0.303311,-0.075745,-0.244403,0.044823,-0.262665,-0.206717,0.025739,-0.278786,-0.067715,0.270636,0.083944,0.291727,-0.105083,0.131594,-0.084821,0.454332,-0.033998
2,0.170616,-0.435246,0.100215,-0.043610,0.330289,0.502139,-0.226096,0.234854,-0.316113,-0.091387,-0.271925,0.111460,-0.611972,-0.076087,0.107744,-0.005868,0.365481,0.176191,-0.519185,0.150880,0.567667,-0.049746,0.220452,-0.135826,0.338678,-0.064376,0.058292,0.146376,0.074083,0.461452,0.372899,0.268020,0.067177,-0.304049,-0.070484,0.348015,-0.228523,0.041331,0.205810,0.440166,...,0.326271,0.417339,0.174676,0.404286,0.205107,-0.412850,0.197703,0.127029,0.210548,-0.287393,-0.018736,-0.104600,-0.209796,-0.345534,0.006915,0.646273,-0.235766,0.334497,0.054455,0.047399,0.297288,-0.118444,0.037223,-0.323689,-0.179704,-0.209463,-0.090725,-0.256350,-0.182799,-0.287376,-0.294561,0.072671,0.127512,0.115997,0.360114,-0.090129,0.050056,-0.022320,0.432490,-0.021742
3,0.112674,-0.106532,0.029696,-0.188042,0.024403,0.845828,-0.371575,0.434558,-0.001989,0.009475,-0.358585,0.081786,-0.709006,-0.042012,-0.072861,0.049609,0.480018,0.533367,-0.708592,0.338966,0.901059,-0.188399,0.295823,-0.069612,0.442792,-0.050615,-0.087286,0.070540,0.349901,0.720822,0.360695,0.262907,0.411075,-0.229887,-0.177007,0.218887,-0.321949,0.203934,0.382982,0.505422,...,0.347870,0.343024,0.249943,0.487751,0.137853,-0.527354,0.218220,0.322993,0.067699,-0.305773,-0.112115,-0.215133,-0.114937,-0.325125,-0.037927,0.538973,-0.173471,0.268805,0.058968,0.047156,0.247988,-0.069754,0.129090,-0.370793,-0.252172,0.019999,0.002496,-0.264802,0.032591,-0.134799,-0.307827,0.178420,0.267530,0.197801,0.301834,0.044331,-0.034822,-0.060457,0.317629,0.018689
4,0.114391,-0.255723,-0.013560,-0.353646,0.287903,0.444778,-0.394945,0.236275,-0.257917,-0.194771,-0.283504,-0.182900,-0.664141,0.246531,0.118883,0.154575,0.502999,0.335900,-0.479125,0.206199,0.599743,0.122603,0.152475,-0.028883,0.567735,-0.132621,0.008428,0.004917,0.321629,0.602193,0.201842,0.336668,0.075853,-0.191282,-0.038348,0.239463,-0.378110,0.131521,0.214341,0.436583,...,0.550671,0.641284,0.642541,0.608518,0.576593,-0.262087,0.270710,-0.085256,0.284999,0.048246,-0.287580,-0.335839,-0.216407,-0.075507,0.087952,0.153029,-0.057662,0.266731,0.244756,-0.019399,0.347825,-0.268087,-0.268205,-0.268714,-0.482283,-0.037456,-0.146997,-0.309631,-0.240904,-0.1989

In [0]:
# 'elmo' -> the weighted sum of the 3 layers, where the weights are trainable. 
# The output embeddings is of shape (10, 211, 1024), as there are 10 docs with max length of 211 words and for each word 1D vector of length 1024 is generated. 
embeddings_elmo = elmo(data_stem[0:10], signature="default", as_dict=True)["elmo"]
embeddings_elmo.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([10, 211, 1024])

In [0]:
# def elmo_vectors(corpus):

#   embeddings = elmo(corpus, signature=”default”, as_dict=True)["default"]

#   with tf.compat.v1.Session() as sess:
#     sess.run(tf.compat.v1.global_variables_initializer())
#     corpus_elm = sess.run(embeddings)
#     return corpus_elm

# Part 4: BERT

In [0]:
uploaded = files.upload()

Saving lemmatized.pkl to lemmatized.pkl


In [0]:
with open('lemmatized.pkl', 'rb') as file:
    data_lemma = pickle.load(file)

print(data_lemma[0])

car wonder enlighten car see day door sport car look late early call bricklin door small addition bumper separate rest body know tellme model engine spec year production car history info funky look car mail thank


In [0]:
# mark the beginning of each setence with [CLS] and the end with [SEP]
sentences = ["[CLS] " + query + " [SEP]" for query in data_lemma]
print(sentences[0])

[CLS] car wonder enlighten car see day door sport car look late early call bricklin door small addition bumper separate rest body know tellme model engine spec year production car history info funky look car mail thank [SEP]


In [0]:
# load tokenizer and then tokenize each sentence
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0]) 


['[CLS]', 'car', 'wonder', 'en', '##light', '##en', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'brick', '##lin', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tell', '##me', 'model', 'engine', 'spec', 'year', 'production', 'car', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', '[SEP]']


We can already see that some words were separated into more than one token (e.g. en, ##light). Tokens with **##** at front are wordpieces
* e.g. the ‘##bed’ token is separate from the ‘bed’ token; the first is used whenever the subword ‘bed’ occurs within a larger word and the second is used explicitly for when the standalone token ‘thing you sleep on’ occurs.

In [0]:
print(len(tokenized_texts[0]))
print(len(tokenized_texts[1]))

42
73


In [0]:
# pad sequences to the same length.
sentences_padded = pad_sequences(tokenized_texts, dtype=object, maxlen= 200, value='[PAD]', truncating="post",padding="post")
print(len(sentences_padded[0]))
print(len(sentences_padded[1]))
sentences_padded[0] # we have 158 '[PAD]' here

200
200


array(['[CLS]', 'car', 'wonder', 'en', '##light', '##en', 'car', 'see',
       'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call',
       'brick', '##lin', 'door', 'small', 'addition', 'bumper',
       'separate', 'rest', 'body', 'know', 'tell', '##me', 'model',
       'engine', 'spec', 'year', 'production', 'car', 'history', 'info',
       'funky', 'look', 'car', 'mail', 'thank', '[SEP]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '

In [0]:
# convert a token string (or a sequence of tokens) in a single integer id (or a sequence of ids), using the vocabulary.
sentences_converted = [tokenizer.convert_tokens_to_ids(s) for s in sentences_padded]
print(sentences_converted[0])

[101, 2482, 4687, 4372, 7138, 2368, 2482, 2156, 2154, 2341, 4368, 2482, 2298, 2397, 2220, 2655, 5318, 4115, 2341, 2235, 2804, 21519, 3584, 2717, 2303, 2113, 2425, 4168, 2944, 3194, 28699, 2095, 2537, 2482, 2381, 18558, 24151, 2298, 2482, 5653, 4067, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [0]:
model = TFBertModel.from_pretrained('bert-base-uncased')

# input_ids (Numpy array or tf.Tensor of shape (batch_size, sequence_length)) - Indices of input sequence tokens in the vocabulary.
input_ids = tf.cast(sentences_converted[0:10], tf.int32)

outputs = model(input_ids)

# last_hidden_state (tf.Tensor of shape (batch_size, sequence_length, hidden_size)):
# Sequence of hidden-states at the output of the last layer of the model.
last_hidden_state = outputs[0]
print(last_hidden_state) # shape=(10, 200, 768)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Tensor("tf_bert_model_2/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0", shape=(10, 200, 768), dtype=float32)


In [0]:
# Computes the mean of elements across dimensions of a tensor.
mean_embeddings = tf.reduce_mean(last_hidden_state, 1)
print(mean_embeddings)

Tensor("Mean:0", shape=(10, 768), dtype=float32)


In [0]:
init = tf.compat.v1.global_variables_initializer()
with tf.compat.v1.Session() as sess: 
  sess.run(init)
  result = sess.run(mean_embeddings)

What does this object contain? How can you use it for further tasks?
* the result is the embeddings for the 10 documents
* classification and clustering algorithms

In [0]:
corpus_bert_df = pd.DataFrame(result)
corpus_bert_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.444188,1.100101,-0.935996,1.245651,0.546947,-0.666012,-1.177713,0.443392,-1.084244,0.929595,-0.777819,-0.210702,-0.423582,1.144433,-0.456767,0.770374,-0.403775,-0.284451,0.302530,-0.561925,-0.252655,0.006825,-1.496661,-0.464272,-1.727918,-0.015069,-0.098422,0.180757,-0.286181,-1.393335,0.100001,0.242093,0.289974,-0.278306,0.667752,-0.013723,0.174256,0.319929,0.466881,0.357222,...,-0.662573,-1.243154,-0.593143,1.617878,-0.412915,-0.621891,2.304423,-1.576978,-0.420551,-0.790983,-0.111788,0.586367,-0.015985,0.567196,-0.948473,0.436093,-0.538246,-0.096155,-0.575459,1.151151,0.396650,-0.018295,0.736898,1.020962,-0.230087,0.407396,0.096953,-0.439079,-0.530780,-0.960460,0.810155,0.020579,0.278628,1.630573,-0.247574,1.537647,0.169610,-0.362706,-0.260953,-0.859586
1,-0.495800,0.953508,-0.738197,1.223702,0.605137,-0.621535,-1.161236,0.407087,-1.007557,1.038661,-0.631434,-0.295988,-0.378191,1.151657,-0.257420,0.949998,-0.396444,-0.304169,0.278960,-0.589447,-0.186228,-0.111663,-1.512228,-0.300386,-1.509118,-0.102638,-0.197310,0.271923,-0.320225,-1.130942,-0.034514,0.138432,0.348741,-0.325967,0.681886,-0.081913,0.169727,0.377128,0.344720,0.252236,...,-0.508013,-1.205985,-0.543007,1.521999,-0.473936,-0.542578,2.100387,-1.550788,-0.467796,-0.781829,-0.119734,0.534113,-0.070046,0.625576,-0.893709,0.533473,-0.749813,-0.178285,-0.571231,1.075778,0.344113,-0.012019,0.586486,1.072090,-0.372748,0.351672,0.016245,-0.457562,-0.511103,-0.976711,0.797093,0.087714,0.289625,1.510937,-0.332220,1.260571,0.029614,-0.205057,-0.262934,-0.853123
2,-0.532105,0.537053,-0.568269,0.911499,0.331941,-0.192685,-1.110590,0.124648,-0.927379,1.164988,-0.173581,-0.457115,-0.248573,1.100899,0.100786,1.331759,-0.364250,-0.344967,0.209359,-0.792178,0.181295,-0.263549,-1.579016,0.131428,-0.820478,-0.326726,-0.510107,0.312708,-0.466829,-0.697966,-0.461463,-0.186646,0.641361,-0.578009,0.872216,-0.186644,0.019475,0.459600,0.124136,0.216154,...,-0.024290,-1.071862,-0.444786,1.127964,-0.362943,-0.277416,1.738291,-1.485155,-0.896855,-0.862652,-0.163784,0.653854,-0.267181,0.542343,-0.711012,0.688035,-1.170877,-0.413913,-0.333103,1.208646,0.194328,-0.032451,-0.011306,1.128178,-0.725309,0.446829,-0.105770,-0.387831,-0.281123,-0.962004,0.726017,0.005293,0.238001,1.163590,-0.456922,0.540424,-0.189006,0.238768,-0.167754,-0.952889
3,-0.440837,0.933942,-0.758160,1.145196,0.466360,-0.460466,-1.206424,0.391579,-0.953852,1.036364,-0.516996,-0.309602,-0.333305,1.169036,-0.210336,0.936873,-0.383129,-0.335380,0.329385,-0.543724,-0.072009,-0.085049,-1.523994,-0.193882,-1.348188,-0.192114,-0.191211,0.252828,-0.301395,-1.106776,-0.124971,0.003373,0.437882,-0.300574,0.680199,-0.189449,0.136030,0.320430,0.210947,0.246213,...,-0.460251,-1.167836,-0.516362,1.445408,-0.352377,-0.491441,2.060647,-1.586345,-0.573098,-0.806642,-0.135708,0.629771,-0.106148,0.589892,-0.863242,0.572267,-0.902633,-0.188690,-0.518460,1.153798,0.318327,0.007267,0.404251,1.073805,-0.379887,0.443521,0.034585,-0.412958,-0.444785,-0.941413,0.730281,-0.037272,0.247019,1.450676,-0.414477,1.172171,-0.017078,-0.101400,-0.178428,-0.950528
4,-0.507163,0.800832,-0.780958,1.075143,0.362284,-0.308851,-1.186050,0.288339,-0.998813,1.038841,-0.480842,-0.284024,-0.340879,1.163532,-0.130667,0.987940,-0.411175,-0.278581,0.353910,-0.605265,0.021365,-0.225672,-1.597453,-0.180698,-1.279584,-0.221157,-0.297173,0.191232,-0.308318,-1.021372,-0.156549,0.000216,0.470551,-0.439197,0.847239,-0.116937,0.115490,0.424850,0.226428,0.208028,...,-0.420521,-1.121290,-0.513357,1.454988,-0.360229,-0.440233,2.054844,-1.513781,-0.610263,-0.779781,-0.110624,0.576265,-0.129894,0.639509,-0.793984,0.606536,-0.851460,-0.176837,-0.535758,1.248909,0.363372,-0.016558,0.407441,1.067623,-0.363996,0.35